# Laboratorio 2 - Reglas de Asociación y Patrones Secuenciales

## Entregable 2 - Patrones Secuenciales

Práctica realizada por: Belén Ortega Pérez, José Ignacio Navas Sanz, Paula Hipola Gómez

Importamos las librerias con las que trabajaremos.

In [1]:
%reset

import numpy as np                        #Esta libreria nos permite trabajar con numericos
import pandas as pd                       #Pandas nos permite trabajar con datasets y csv
import matplotlib.pyplot as plt           #Las dos librerias anteriores nos permiten pintar las funciones
from sklearn import preprocessing         #Nos sirve para normalizar los datos

from gsppy.gsp import GSP                 #Librería para poder implementar Generalized Sequential Patterns
from mlxtend.frequent_patterns import apriori #Libería relacionada con A priori

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


### 1. El conjunto de datos spotify_top10.csv contiene el ranking diario de las 10 canciones más reproducidas en Spotify cada día por país. Analizando esta información se pueden extraer conclusiones sobre cómo evoluciona la popularidad de las canciones. Este estudio se puede llevar a cabo aplicando el algoritmo Generalized Sequential Patterns utilizando la implementación de éste disponible en la librería gsppy. 
#### Prueba al menos dos configuraciones de soporte diferentes. Para una de ellas, interpreta algunos de los patrones secuenciales que te resulten curiosos. 

Añadimos el dataset con el que vamos a trabajar. En este caso vamos a utilizar una colección de datos que contiene el top 10 canciones escuchadas cada día por país. 

In [2]:
data = pd.read_csv('spotify_top10.csv')
data.head()                               #Función para ver las primeras instancias del dataset

,Position,Track Name,Artist,Streams,URL,Date,Region
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,2017-01-01,ec
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,2017-01-01,ec
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,2017-01-01,ec
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,2017-01-01,ec
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,2017-01-01,ec


Lo primero que debemos llevar a cabo es una limpieza del dataset. Para ello, primero averiguaremos cuántos missing values y valores duplicados tiene. 

**Datos duplicados**

In [3]:
duplicates_data = data.duplicated().sum()   #Cuenta el número de datos duplicados
print('Hay',duplicates_data,'datos duplicados')
data.drop_duplicates(inplace = True)

Hay 0 datos duplicados


**Missing values**

In [4]:
missing_data1 = data.isnull().sum().sum()  #Cuenta el total de missing values en el dataset
missing_data2 = data.isnull().sum()        #Cuenta el total de missing values por atributo
print('Hay',missing_data1,' valores nulos')
print('\nDistribuido en columnas:')
print(missing_data2)

Hay 354  valores nulos

Distribuido en columnas:
Position        0
Track Name    177
Artist        177
Streams         0
URL             0
Date            0
Region          0
dtype: int64


In [5]:
# En los campos donde el Track Name es nulo, copiamos la URL.
data.loc[(data["Track Name"].isnull())&(data["URL"].notnull()),"Track Name"] = data["URL"]
missing_data2 = data.isnull().sum() 
print(missing_data2)

#MIRAR ESTO
for i in data:
    comparison = np.where(data["Track Name"] == data["URL"])
    if(comparison is True):
        print(data.loc[[i]])

Position        0
Track Name      0
Artist        177
Streams         0
URL             0
Date            0
Region          0
dtype: int64


In [6]:
fechas = data['Date']

fechas = np.sort(fechas.unique())
totalCanciones_dia = 0

ranking_array = []


for fecha in fechas:
    rankingPorDia = data[data['Position'] , data['URL'] , (data['Date'] == fecha)]
    totalCanciones_dia += rankingPorDia.shape[0]
    ranking_array.append(rankingPorDia.shape[0])
    fecha = pd.DatetimeIndex(data.Date) + pd.DateOffset(1)
    
print(totalCanciones_dia)

TypeError: '(0          1
1          2
2          3
3          4
4          5
          ..
196680     6
196681     7
196682     8
196683     9
196684    10
Name: Position, Length: 196685, dtype: int64, 0         https://open.spotify.com/track/3AEZUABDXNtecAO...
1         https://open.spotify.com/track/6mICuAdrwEjh6Y6...
2         https://open.spotify.com/track/3QwBODjSEzelZyV...
3         https://open.spotify.com/track/7DM4BPaS7uofFul...
4         https://open.spotify.com/track/6rQSrBHf7HlZjtc...
                                ...                        
196680    https://open.spotify.com/track/5k38wzpLb15Ygnc...
196681    https://open.spotify.com/track/5yFSF6qQA1rcLsQ...
196682    https://open.spotify.com/track/3hBBKuWJfxlIlnd...
196683    https://open.spotify.com/track/3VSFri2BxieCG7R...
196684    https://open.spotify.com/track/53HxUdYRCRMdzYP...
Name: URL, Length: 196685, dtype: object, 0          True
1          True
2          True
3          True
4          True
          ...  
